In [1]:
import logging
import sys, os
import time
import warnings
import torch
import argparse
from pathlib import Path
import numpy as np
from datetime import datetime
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import MSELoss, L1Loss
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.serialization import save
from gnn.model.metric import EarlyStopping
from gnn.model.gated_solv_network import GatedGCNSolvationNetwork, InteractionMap, SelfInteractionMap
from gnn.data.dataset import SolvationDataset, train_validation_test_split, solvent_split, element_split, substructure_split, stratified_solvent_split, stratified_split
from gnn.data.dataloader import DataLoaderSolvation
from gnn.data.grapher import HeteroMoleculeGraph
from gnn.data.featurizer import (
    SolventAtomFeaturizer,
    BondAsNodeFeaturizerFull,
    SolventGlobalFeaturizer,
)
from gnn.data.solvent_graph import HeteroMoleculeGraph2
from gnn.data.dataset import load_mols_labels
from gnn.utils import (
    load_checkpoints,
    pickle_load,
    save_checkpoints,
    seed_torch,
    pickle_dump,
    yaml_dump,
)
from sklearn.metrics import mean_squared_error

ls=[]
p=[]
def parse_args():
    parser = argparse.ArgumentParser(description="GatedSolvationNetwork")

    # input files and global variables
    parser.add_argument('--dataset-file', type=str, default="/home/1739678401/2024.9dataclean/MMGNN_0901/data/exp_data.csv")
    parser.add_argument('--dataset-pickle', type=str, default="/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer/graphers.pkl")
    parser.add_argument('--dielectric-constants', type=str, default=None)
    parser.add_argument('--molecular-refractivity', type=bool, default=False)
    parser.add_argument('--molecular-volume', type=bool, default=False)

    # output dir
    parser.add_argument('--save-dir', type=str, default="/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer")
    parser.add_argument("--load-dataset", type=int, default=0, help="read dataset")
    # training params
    parser.add_argument('--random-seed', type=int, default=50)
    parser.add_argument('--feature-scaling', type=bool, default=True)
    parser.add_argument('--solvent-split', type=str, default=None)
    parser.add_argument('--solvent-stratified-split', type=str, default=None)
    parser.add_argument('--solvent-stratified-frac', type=float, default=0.1)
    parser.add_argument('--stratified-split', type=bool, default=False)
    parser.add_argument('--element-split', type=str, default=None)
    parser.add_argument('--scaffold-split', type=bool, default=False)
    parser.add_argument('--attention-map', type=str, default=None)
    parser.add_argument('--partial-charges', type=str, default=None)


    # embedding layer
    parser.add_argument("--embedding-size", type=int, default=48)

    # gated layer
    parser.add_argument("--gated-num-layers", type=int, default=3)
    parser.add_argument("--gated-hidden-size", type=int, nargs="+", default=[1600])
    parser.add_argument("--gated-num-fc-layers", type=int, default=3)
    parser.add_argument("--gated-graph-norm", type=int, default=0)
    parser.add_argument("--gated-batch-norm", type=int, default=0)
    parser.add_argument("--gated-activation", type=str, default="LeakyReLU")
    parser.add_argument("--gated-residual", type=int, default=1)
    parser.add_argument("--gated-dropout", type=float, default=0.0)

    # readout layer
    parser.add_argument(
        "--num-lstm-iters",
        type=int,
        default=6,
        help="number of iterations for the LSTM in set2set readout layer",
    )
    parser.add_argument(
        "--num-lstm-layers",
        type=int,
        default=3,
        help="number of layers for the LSTM in set2set readout layer",
    )

    # fc layer
    parser.add_argument("--fc-num-layers", type=int, default=4)
    parser.add_argument("--fc-hidden-size", type=int, nargs="+", default=[1400])
    parser.add_argument("--fc-batch-norm", type=int, default=0)
    parser.add_argument("--fc-activation", type=str, default="LeakyReLU")
    parser.add_argument("--fc-dropout", type=float, default=0.5)

    # training
    parser.add_argument("--start-epoch", type=int, default=1)
    parser.add_argument("--epochs", type=int, default=1000, help="number of epochs")
    parser.add_argument("--batch-size", type=int, default=64, help="batch size")
    parser.add_argument("--lr", type=float, default=0.0001, help="learning rate")
    parser.add_argument("--weight-decay", type=float, default=0.0, help="weight decay")
    parser.add_argument("--restore", type=int, default=0, help="read checkpoints")
    parser.add_argument(
        "--dataset-state-dict-filename", type=str, default="dataset_state_dict.pkl"
    )
    # gpu
    parser.add_argument(
        "--gpu", type=int, default=3, help="GPU index. None to use CPU."
    )

    parser.add_argument(
        "--distributed",
        type=int,
        default=0,
        help="DDP training, --gpu is ignored if this is True",
    )
    parser.add_argument(
        "--num-gpu",
        type=int,
        default=1,
        help="Number of GPU to use in distributed mode; ignored otherwise.",
    )
    parser.add_argument(
        "--dist-url",
        default="tcp://localhost:13456",
        type=str,
        help="url used to set up distributed training",
    )
    
    parser.add_argument("--dist-backend", type=str, default="nccl")

    # output file (needed by hypertunity)
    parser.add_argument("--output_file", type=str, default="results.pkl")

    args = parser.parse_args(args=[])
    if len(args.gated_hidden_size) == 1:
        args.gated_hidden_size = args.gated_hidden_size * args.gated_num_layers
    else:
        assert len(args.gated_hidden_size) == args.gated_num_layers, (
            "length of `gat-hidden-size` should be equal to `num-gat-layers`, but got "
            "{} and {}.".format(args.gated_hidden_size, args.gated_num_layers)
        )

    if len(args.fc_hidden_size) == 1:
        val = 2 * args.gated_hidden_size[-1]
        args.fc_hidden_size = [max(val // 2 ** i, 8) for i in range(args.fc_num_layers)]
    else:
        assert len(args.fc_hidden_size) == args.fc_num_layers, (
            "length of `fc-hidden-size` should be equal to `num-fc-layers`, but got "
            "{} and {}.".format(args.fc_hidden_size, args.fc_num_layers)
        )
    return args

def train(optimizer, model, nodes,nodes1, data_loader, loss_fn, metric_fn, device=None):
    """
    Args:
        metric_fn (function): the function should be using a `sum` reduction method.
    """

    model.train()

    epoch_loss = 0.0
    accuracy = 0.0
    count = 0.0

    for it, (solute_batched_graph, solvent_batched_graph, label) in enumerate(data_loader):
        solute_feats = {nt: solute_batched_graph.nodes[nt].data["feat"] for nt in nodes1}
        solvent_feats = {nt: solvent_batched_graph.nodes[nt].data["feat"] for nt in nodes}
        target = torch.squeeze(label["value"])
        solute_norm_atom = label["solute_norm_atom"]
        solute_norm_bond = label["solute_norm_bond"]
        solvent_norm_atom = label["solvent_norm_atom"]
        solvent_norm_bond = label["solvent_norm_bond"]

        if device is not None:
            solute_feats = {k: v.to(device) for k, v in solute_feats.items()}
            solvent_feats = {k: v.to(device) for k, v in solvent_feats.items()}
            target = target.to(device)
            solute_norm_atom = solute_norm_atom.to(device)
            solute_norm_bond = solute_norm_bond.to(device)
            solvent_norm_atom = solvent_norm_atom.to(device)
            solvent_norm_bond = solvent_norm_bond.to(device)
        pred,_ = model(solute_batched_graph, solvent_batched_graph, solute_feats, 
                     solvent_feats, solute_norm_atom, solute_norm_bond, 
                     solvent_norm_atom, solvent_norm_bond)

        pred = pred.view(-1)
        target = target.view(-1)
        loss = loss_fn(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.detach().item()
        accuracy += metric_fn(pred, target).detach().item()
        count += len(target)
    
    epoch_loss /= it + 1
    accuracy /= count

    return epoch_loss, accuracy

def evaluate(model, nodes,nodes1, data_loader, metric_fn, scaler = None, device=None, return_preds=False):
    """
    Evaluate the accuracy of a validation set of test set.
    Args:
        metric_fn (function): the function should be using a `sum` reduction method.
    """
    model.eval()
    with torch.no_grad():
        accuracy = 0.0
        count = 0.0

        preds = []
        y_true = []

        for solute_batched_graph, solvent_batched_graph, label in data_loader:
            solute_feats = {nt: solute_batched_graph.nodes[nt].data["feat"] for nt in nodes1}
            solvent_feats = {nt: solvent_batched_graph.nodes[nt].data["feat"] for nt in nodes}
            target = torch.squeeze(label["value"])
            #stdev = label["scaler_stdev"]
            solvent_norm_atom = label["solvent_norm_atom"]
            solvent_norm_bond = label["solvent_norm_bond"]
            solute_norm_atom = label["solute_norm_atom"]
            solute_norm_bond = label["solute_norm_bond"]

            if device is not None:
                solute_feats = {k: v.to(device) for k, v in solute_feats.items()}
                solvent_feats = {k: v.to(device) for k, v in solvent_feats.items()}
                target = target.to(device)
                solute_norm_atom = solute_norm_atom.to(device)
                solute_norm_bond = solute_norm_bond.to(device)
                solvent_norm_atom = solvent_norm_atom.to(device)
                solvent_norm_bond = solvent_norm_bond.to(device)

            pred,loss_1 = model(solute_batched_graph, solvent_batched_graph, solute_feats, 
                     solvent_feats, solute_norm_atom, solute_norm_bond, 
                     solvent_norm_atom, solvent_norm_bond)
            pred = pred.view(-1)
            target = target.view(-1)

            if scaler is not None:
                pred = scaler.inverse_transform(pred.cpu())
                pred = pred.to(device)

            accuracy += metric_fn(pred, target).detach().item()
            count += len(target)

            batch_pred = pred.tolist()
            batch_target = target.tolist()
            preds.extend(batch_pred)
            y_true.extend(batch_target)

    if return_preds:
        return y_true, preds

    else:
        return accuracy / count

def grapher(dielectric_constant=None, mol_volume=False, mol_refract=False, partial_charges=None,lable=False):
    atom_featurizer = SolventAtomFeaturizer(partial_charges=partial_charges)
    bond_featurizer = BondAsNodeFeaturizerFull(length_featurizer=None, dative=False)
    global_featurizer = SolventGlobalFeaturizer(dielectric_constant=dielectric_constant, mol_volume=mol_volume, mol_refract=mol_refract)
    if lable:
        grapher = HeteroMoleculeGraph(atom_featurizer, bond_featurizer, global_featurizer, self_loop=True)
    else:
        grapher = HeteroMoleculeGraph2(atom_featurizer, bond_featurizer, global_featurizer, self_loop=True)

    return grapher


/home/1739678401/anaconda3/envs/dataclean/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch


In [4]:
args = parse_args()
print(args)

if args.save_dir is not None:
    os.makedirs(args.save_dir, exist_ok=True)

logging.basicConfig(
filename=os.path.join(args.save_dir, '{}.log'.format(
    datetime.now().strftime("gnn_%Y_%m_%d-%I_%M_%p"))),
format="%(asctime)s:%(name)s:%(levelname)s: %(message)s",
level=logging.INFO,
)

Namespace(attention_map=None, batch_size=64, dataset_file='/home/1739678401/2024.9dataclean/MMGNN_0901/data/exp_data.csv', dataset_pickle='/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer/graphers.pkl', dataset_state_dict_filename='dataset_state_dict.pkl', dielectric_constants=None, dist_backend='nccl', dist_url='tcp://localhost:13456', distributed=0, element_split=None, embedding_size=48, epochs=1000, fc_activation='LeakyReLU', fc_batch_norm=0, fc_dropout=0.5, fc_hidden_size=[3200, 1600, 800, 400], fc_num_layers=4, feature_scaling=True, gated_activation='LeakyReLU', gated_batch_norm=0, gated_dropout=0.0, gated_graph_norm=0, gated_hidden_size=[1600, 1600, 1600], gated_num_fc_layers=3, gated_num_layers=3, gated_residual=1, gpu=3, load_dataset=0, lr=0.0001, molecular_refractivity=False, molecular_volume=False, num_gpu=1, num_lstm_iters=6, num_lstm_layers=3, output_file='results.pkl', partial_charges=None, random_seed=50, restore=0, save_dir='/home/1739678401/2024.9dataclean/MM

In [5]:
random_seed = args.random_seed
world_size=None
seed_torch(random_seed)

train

In [ ]:
if not args.distributed or (args.distributed and args.gpu == 0):
    print("\n\nStart training at: ", datetime.now())

if args.save_dir is None:
    args.save_dir = os.getcwd()

if args.distributed:
    dist.init_process_group(
        args.dist_backend,
        init_method = args.dist_url,
        world_size = world_size,
        rank = args.gpu
    )

if args.restore:
    dataset_state_dict_filename = args.dataset_state_dict_filename

    if dataset_state_dict_filename is None:
        warnings.warn("Restore with `args.dataset_state_dict_filename` set to None.")
    elif not Path(dataset_state_dict_filename).exists():
        warnings.warn(
            f"`{dataset_state_dict_filename} not found; set "
            f"args.dataset_state_dict_filename` to None"
        )
        dataset_state_dict_filename = None
else:
    dataset_state_dict_filename = None

# Load molecules and labels from file
mols, labels = load_mols_labels(args.dataset_file)

if args.load_dataset:
    data_dict = args.dataset_pickle
    dataset = pickle_load(data_dict)

else:
    if args.dielectric_constants is not None:
        dc_file = Path(args.dielectric_constants)        
        dataset = SolvationDataset(
            solute_grapher = grapher(mol_volume = args.molecular_volume,
                                    mol_refract = args.molecular_refractivity,
                                    partial_charges=args.partial_charges),
            solvent_grapher = grapher(dielectric_constant=True,
                                    mol_volume = args.molecular_volume,
                                    mol_refract = args.molecular_refractivity,
                                    partial_charges=args.partial_charges),
            molecules = mols,
            labels = labels,
            solute_extra_features = None,
            solvent_extra_features=dc_file,
            feature_transformer = False,
            label_transformer= False,
            state_dict_filename=dataset_state_dict_filename)

    else:
        dataset = SolvationDataset(
            solute_grapher = grapher(mol_volume=args.molecular_volume, mol_refract = args.molecular_refractivity, partial_charges=args.partial_charges),
            solvent_grapher = grapher(mol_volume=args.molecular_volume, mol_refract = args.molecular_refractivity, partial_charges=args.partial_charges,lable=True),
            molecules = mols,
            labels = labels,
            solute_extra_features = None,
            solvent_extra_features = None,
            feature_transformer = False,
            label_transformer= False,
            state_dict_filename=dataset_state_dict_filename)
pickle_dump([dataset.solute_grapher, dataset.solvent_grapher], os.path.join(args.save_dir,"graphers.pkl"))
# Save the solute and solvent graphers for loading datasets later




Start training at:  2025-05-07 15:15:23.075625


[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not removing hydrogen atom without neighbors
[15:15:23] WARNING: not r

len(self._species)=12,type(self._species)=<class 'list'>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
2

In [ ]:
pickle_dump(dataset,os.path.join(args.save_dir,"dataset.pkl"))

In [ ]:
best = np.finfo(np.float32).max
os.makedirs(args.save_dir, exist_ok=True)


trainset, valset, testset = train_validation_test_split(
    dataset, validation=0.1, test=0.1, random_seed=args.random_seed)

# Scale training dataset features
if args.feature_scaling:
    solute_features_scaler,solvent_features_scaler= trainset.normalize_features()
    #solute_features_scaler, solvent_features_scaler = trainset.normalize_features()
    valset.normalize_features(solute_features_scaler, solvent_features_scaler)
    testset.normalize_features(solute_features_scaler, solvent_features_scaler)
    #testset.normalize_features(solute_features_scaler)
else:
    solute_features_scaler, solvent_features_scaler = None, None

label_scaler = trainset.normalize_labels()
if not args.distributed or (args.distributed and args.gpu == 0):
    torch.save(dataset.state_dict(), os.path.join(args.save_dir, args.dataset_state_dict_filename))
    print(
        "Trainset size: {}, valset size: {}: testset size: {}.".format(
            len(trainset), len(valset), len(testset)
        )
    )
if args.distributed:
    train_sampler = torch.utils.data.distributed.DistributedSampler(trainset)
else:
    train_sampler = None

train_loader = DataLoaderSolvation(
    trainset,
    batch_size = args.batch_size,
    shuffle = (train_sampler is None),
    sampler = train_sampler
)
# larger val and test set batch_size is faster but needs more memory
# adjust the batch size of val and test set to fit memory
bs = max(len(valset) // 10, 1)
val_loader = DataLoaderSolvation(valset, batch_size=bs, shuffle=False)
bs = max(len(testset) // 10, 1)
test_loader = DataLoaderSolvation(testset, batch_size=bs, shuffle=False)
### model
feature_names = ["atom", "bond", "global"]
solute_feature_names = ["atom", "bond","atom2", "bond2", "global"]
set2set_ntypes_direct = ["global"]
solute_feature_size = dataset.feature_sizes[0]
# solute_feature_size ={'bond': 11,'atom': 28,'atom2': 28, 'bond2': 11, 'global': 3}
solute_feature_size = {'bond': 11, 'atom': 30, 'atom2': 30, 'bond2': 11, 'global': 3}

solvent_feature_size = dataset.feature_sizes[1]
args.solute_feature_size = solute_feature_size
args.solvent_feature_size = solvent_feature_size
args.set2set_ntypes_direct = set2set_ntypes_direct


Transfer Learning

In [6]:
model = GatedGCNSolvationNetwork(
        solute_in_feats=args.solute_feature_size,
        solvent_in_feats=args.solvent_feature_size,
        embedding_size=args.embedding_size,
        gated_num_layers=args.gated_num_layers,
        gated_hidden_size=args.gated_hidden_size,
        gated_num_fc_layers=args.gated_num_fc_layers,
        gated_graph_norm=args.gated_graph_norm,
        gated_batch_norm=args.gated_batch_norm,
        gated_activation=args.gated_activation,
        gated_residual=args.gated_residual,
        gated_dropout=args.gated_dropout,
        num_lstm_iters=args.num_lstm_iters,
        num_lstm_layers=args.num_lstm_layers,
        set2set_ntypes_direct=args.set2set_ntypes_direct,
        fc_num_layers=args.fc_num_layers,
        fc_hidden_size=args.fc_hidden_size,
        fc_batch_norm=args.fc_batch_norm,
        fc_activation=args.fc_activation,
        fc_dropout=args.fc_dropout,
        outdim=1,
        conv="GatedGCNConv",
    )
optimizer = torch.optim.Adam(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay
)
loss_func = MSELoss(reduction="mean")
metric = L1Loss(reduction="sum")

scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.4, patience=50, verbose=True
)
stopper = EarlyStopping(patience=150)

state_dict_objs = {"model": model, "optimizer": optimizer, "scheduler": scheduler}
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
    state_dict_objs, map_location=loc, save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/exp", filename="best_checkpoint.pkl"
)
for name, param in model.named_parameters():
    if not any(layer_name in name for layer_name in ['fc_layers']):
        param.requires_grad = False
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)       

AttributeError: 'Namespace' object has no attribute 'solute_feature_size'

In [ ]:
model.to(args.gpu)
print(args.gpu)

In [ ]:

if not args.distributed or (args.distributed and args.gpu == 0):
    print("\n\n# Epoch     Loss         TrainAcc        ValAcc     Time (s)")
    sys.stdout.flush()
for epoch in range(args.start_epoch, args.epochs):
    ti = time.time()
    # In distributed mode, calling the set_epoch method is needed to make shuffling
    # work; each process will use the same random seed otherwise.
    if args.distributed:
        train_sampler.set_epoch(epoch)
    # train
    loss, train_acc = train(
        optimizer, model, feature_names, solute_feature_names,train_loader, loss_func, metric, args.gpu)
    # bad, we get nan
    if np.isnan(loss):
        print("\n\nBad, we get nan for loss. Exiting")
        sys.stdout.flush()
        sys.exit(1)
    # evaluate
    val_acc = evaluate(model, feature_names, solute_feature_names,val_loader, metric, label_scaler, args.gpu)
    if stopper.step(val_acc):
        pickle_dump(best, os.path.join(args.save_dir, args.output_file))  # save results for hyperparam tune
        break
    scheduler.step(val_acc)
    is_best = val_acc < best
    if is_best:
        best = val_acc
    # save checkpoint
    if not args.distributed or (args.distributed and args.gpu == 0):
        misc_objs = {"best": best, "epoch": epoch}
        scaler_objs = {'label_scaler': {
                        'means': label_scaler.mean,
                        'stds': label_scaler.std
                        } if label_scaler is not None else None,
                        'solute_features_scaler': {
                        'means': solute_features_scaler.mean,
                        'stds': solute_features_scaler.std
                        } if solute_features_scaler is not None else None,
                        'solvent_features_scaler': {
                        'means': solvent_features_scaler.mean,
                        'stds': solvent_features_scaler.std
                        } if solvent_features_scaler is not None else None}
        save_checkpoints(
            state_dict_objs,
            misc_objs,
            scaler_objs,
            is_best,
            msg=f"epoch: {epoch}, score {val_acc}",
            save_dir=args.save_dir)
        tt = time.time() - ti
        print(
            "{:5d}   {:12.6e}   {:12.6e}   {:12.6e}   {:.2f}".format(
                epoch, loss, train_acc, val_acc, tt
            )
        )
        ls.append( val_acc)
        if epoch % 10 == 0:
            sys.stdout.flush()

test

In [6]:
#del dataset
dataset = pickle_load("/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer/dataset.pkl")

In [7]:
import pandas as pd
df = pd.read_csv("/home/1739678401/2024.9dataclean/MMGNN_0901/top_1000_indices.csv")
index_list = df['Index'].tolist()
print(index_list)
print(len(index_list))


[4313, 4316, 4327, 4461, 4372, 4366, 4373, 4457, 4364, 4338, 4439, 4315, 4321, 4318, 4438, 4440, 4319, 4385, 4410, 4322, 4375, 4437, 4398, 4400, 4325, 4442, 4307, 4363, 4389, 4344, 4369, 4348, 4444, 4387, 4378, 4396, 4399, 4458, 4336, 4416, 4350, 4380, 4446, 4395, 4383, 4342, 4393, 4391, 4402, 4360, 4412, 4419, 4421, 4382, 4404, 4310, 4371, 4414, 4306, 4397, 4311, 4452, 4436, 4428, 4435, 4447, 4454, 4434, 4423, 4433, 4448, 4456, 4431, 4425, 4312, 4405, 4430, 4450, 4451, 4407, 4429, 4424, 4453, 4406, 4455, 4459, 4652, 4654, 8333, 8330, 3273, 8344, 6419, 2321, 8340, 8342, 8335, 8331, 4512, 4992, 2319, 7300, 8339, 8329, 6780, 8334, 4314, 8336, 8324, 2318, 6248, 1621, 8343, 8337, 4873, 8316, 8318, 8325, 7296, 8321, 2317, 2322, 5707, 3286, 2342, 8345, 7279, 8317, 8322, 1619, 7265, 7267, 7297, 7269, 8332, 4653, 2316, 8338, 7289, 2558, 3289, 3291, 3293, 7275, 2341, 2343, 6249, 1623, 7278, 7282, 8319, 8323, 7266, 7290, 8320, 2315, 2320, 7294, 3412, 3284, 2561, 2563, 2565, 3290, 3292, 3278, 234

In [8]:
from gnn.data.dataset import Subset
def gettest(dataset,indices):
    """
    Split a dataset into training, validation, and test set.
    The training set will be automatically determined based on `validation` and `test`,
    i.e. train = 1 - validation - test.
    Args:
        dataset: the dataset
        validation (float, optional): The amount of data (fraction) to be assigned to
            validation set. Defaults to 0.1.
        test (float, optional): The amount of data (fraction) to be assigned to test
            set. Defaults to 0.1.
        random_seed (int, optional): random seed that determines the permutation of the
            dataset. Defaults to 0.
    Returns:
        [train set, validation set, test_set]
    """
    test_idx = indices
    return Subset(dataset, test_idx)


In [9]:
trainset, valset, testset = train_validation_test_split(
    dataset, validation=0.1, test=0.1, random_seed=args.random_seed)
testset=gettest(dataset,index_list)

In [ ]:

if args.feature_scaling:
    solute_features_scaler,solvent_features_scaler= trainset.normalize_features()
    #solute_features_scaler, solvent_features_scaler = trainset.normalize_features()
    valset.normalize_features(solute_features_scaler, solvent_features_scaler)
    testset.normalize_features(solute_features_scaler, solvent_features_scaler)
    #testset.normalize_features(solute_features_scaler)
else:
    solute_features_scaler, solvent_features_scaler = None, None

label_scaler = trainset.normalize_labels()
if not args.distributed or (args.distributed and args.gpu == 0):
    torch.save(dataset.state_dict(), os.path.join(args.save_dir, args.dataset_state_dict_filename))
    print(
        "Trainset size: {}, valset size: {}: testset size: {}.".format(
            len(trainset), len(valset), len(testset)
        )
    )
if args.distributed:
    train_sampler = torch.utils.data.distributed.DistributedSampler(trainset)
else:
    train_sampler = None

train_loader = DataLoaderSolvation(
    trainset,
    batch_size = args.batch_size,
    shuffle = (train_sampler is None),
    sampler = train_sampler
)
# larger val and test set batch_size is faster but needs more memory
# adjust the batch size of val and test set to fit memory
bs = max(len(testset) // 10, 1)
test_loader = DataLoaderSolvation(testset, batch_size=bs, shuffle=False)
### model
feature_names = ["atom", "bond", "global"]
solute_feature_names = ["atom", "bond","atom2", "bond2", "global"]
set2set_ntypes_direct = ["global"]
solute_feature_size = dataset.feature_sizes[0]
# solute_feature_size ={'bond': 11,'atom': 28,'atom2': 28, 'bond2': 11, 'global': 3}
solute_feature_size = {'bond': 11, 'atom': 30, 'atom2': 30, 'bond2': 11, 'global': 3}

solvent_feature_size = dataset.feature_sizes[1]
args.solute_feature_size = solute_feature_size
args.solvent_feature_size = solvent_feature_size
args.set2set_ntypes_direct = set2set_ntypes_direct


Trainset size: 7024, valset size: 878: testset size: 1000.


In [11]:
model = GatedGCNSolvationNetwork(
        solute_in_feats=args.solute_feature_size,
        solvent_in_feats=args.solvent_feature_size,
        embedding_size=args.embedding_size,
        gated_num_layers=args.gated_num_layers,
        gated_hidden_size=args.gated_hidden_size,
        gated_num_fc_layers=args.gated_num_fc_layers,
        gated_graph_norm=args.gated_graph_norm,
        gated_batch_norm=args.gated_batch_norm,
        gated_activation=args.gated_activation,
        gated_residual=args.gated_residual,
        gated_dropout=args.gated_dropout,
        num_lstm_iters=args.num_lstm_iters,
        num_lstm_layers=args.num_lstm_layers,
        set2set_ntypes_direct=args.set2set_ntypes_direct,
        fc_num_layers=args.fc_num_layers,
        fc_hidden_size=args.fc_hidden_size,
        fc_batch_norm=args.fc_batch_norm,
        fc_activation=args.fc_activation,
        fc_dropout=args.fc_dropout,
        outdim=1,
        conv="GatedGCNConv",
    )
optimizer = torch.optim.Adam(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay
)
loss_func = MSELoss(reduction="mean")
metric = L1Loss(reduction="sum")

scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.4, patience=50, verbose=True
)
stopper = EarlyStopping(patience=150)

state_dict_objs = {"model": model, "optimizer": optimizer, "scheduler": scheduler}
model.to(args.gpu)

GatedGCNSolvationNetwork(
  (solute_embedding): UnifySize(
    (linears): ModuleDict(
      (bond): Linear(in_features=11, out_features=48, bias=False)
      (atom): Linear(in_features=30, out_features=48, bias=False)
      (atom2): Linear(in_features=30, out_features=48, bias=False)
      (bond2): Linear(in_features=11, out_features=48, bias=False)
      (global): Linear(in_features=3, out_features=48, bias=False)
    )
  )
  (gated_layers): ModuleList(
    (0): GatedGCNConv(
      (activation): LeakyReLU(negative_slope=0.01)
      (A): LinearN(
        (fc_layers): ModuleList(
          (0): Linear(in_features=48, out_features=1600, bias=True)
          (1): LeakyReLU(negative_slope=0.01)
          (2): Linear(in_features=1600, out_features=1600, bias=True)
          (3): LeakyReLU(negative_slope=0.01)
          (4): Linear(in_features=1600, out_features=1600, bias=True)
          (5): Identity()
        )
      )
      (B): LinearN(
        (fc_layers): ModuleList(
          (0): Li

pre_trained

In [ ]:
args.save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer"
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
        state_dict_objs, map_location=loc, save_dir=args.save_dir, filename="best_checkpoint.pkl")

if not args.distributed or (args.distributed and args.gpu == 0):
    test_acc = evaluate(model, feature_names,solute_feature_names, test_loader, metric, label_scaler, args.gpu)
    y_true, y_pred = evaluate(model, feature_names,solute_feature_names, test_loader, metric, 
                                label_scaler, args.gpu, return_preds=True)
    
    print(len(y_true))
    print(len(y_pred))
    print("\n#Test MAE: {:12.6e} \n".format(test_acc))
    print("\n#Test RMSE: {:12.6e} \n".format(mean_squared_error(y_true, y_pred, squared=False)))

theory

In [ ]:
args.save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/theory_random"
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
        state_dict_objs, map_location=loc, save_dir=args.save_dir, filename="best_checkpoint.pkl")

if not args.distributed or (args.distributed and args.gpu == 0):
    test_acc = evaluate(model, feature_names,solute_feature_names, test_loader, metric, label_scaler, args.gpu)
    y_true, y_pred = evaluate(model, feature_names,solute_feature_names, test_loader, metric, 
                                label_scaler, args.gpu, return_preds=True)
    
    print(len(y_true))
    print(len(y_pred))
    print("\n#Test MAE: {:12.6e} \n".format(test_acc))
    print("\n#Test RMSE: {:12.6e} \n".format(mean_squared_error(y_true, y_pred, squared=False)))
    print("\nFinish training at:", datetime.now())
    p.append(mean_squared_error(y_true, y_pred, squared=False))
    results_dict = {'y_true': y_true, 'y_pred': y_pred}
    pickle_dump(results_dict, os.path.join(args.save_dir, f'seed_{random_seed}_test_results.pkl'))

exp

In [ ]:
args.save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/exp"
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
        state_dict_objs, map_location=loc, save_dir=args.save_dir, filename="best_checkpoint.pkl")
label_scaler = testset.normalize_labels()
#test_acc = evaluate(model, feature_names,solute_feature_names, test_loader, metric, label_scaler, args.gpu)
y_true, y_pred = evaluate(model, feature_names,solute_feature_names, test_loader, metric, 
                        label_scaler, args.gpu, return_preds=True)

print(len(y_true))
print(len(y_pred))
mae = np.mean(np.abs(np.array(y_true) - np.array(y_pred)))
print("MAE:", mae)
print("\n#Test RMSE: {:12.6e} \n".format(mean_squared_error(y_true, y_pred, squared=False)))

DGLError: [16:44:54] /opt/dgl/src/runtime/cuda/cuda_device_api.cc:97: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading: CUDA: out of memory
Stack trace:
  [bt] (0) /home/1739678401/anaconda3/envs/dataclean/lib/python3.7/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x4f) [0x1554193324df]
  [bt] (1) /home/1739678401/anaconda3/envs/dataclean/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::CUDADeviceAPI::AllocDataSpace(DLContext, unsigned long, unsigned long, DLDataType)+0xfc) [0x155419b96abc]
  [bt] (2) /home/1739678401/anaconda3/envs/dataclean/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::NDArray::Empty(std::vector<long, std::allocator<long> >, DLDataType, DLContext)+0x351) [0x155419a52831]
  [bt] (3) /home/1739678401/anaconda3/envs/dataclean/lib/python3.7/site-packages/dgl/libdgl.so(dgl::runtime::NDArray::CopyTo(DLContext const&) const+0xc0) [0x155419a89a30]
  [bt] (4) /home/1739678401/anaconda3/envs/dataclean/lib/python3.7/site-

In [17]:
y_true_300 = y_true[:300]
y_pred_300 = y_pred[:300]

mae = mae = np.mean(np.abs(np.array(y_true_300) - np.array(y_pred_300)))
rmse = mean_squared_error(y_true_300, y_pred_300, squared=False)  # squared=False gives RMSE

print("MAE (前300):", mae)
print("RMSE (前300):", rmse)

MAE (前300): 2.537457289665218
RMSE (前300): 3.7795436474054296


In [14]:
for i in range(1000):
    print(y_true[i],y_pred[i])

-1.574657678604126 -2.0406779457127473
-14.600000381469727 -2.690680252708357
-0.927505373954773 -1.9349292173250174
-1.6580196619033813 -1.6438285402868176
-13.479999542236328 -4.309522964113922
-2.057279586791992 -2.7806853792048036
-1.820356011390686 -2.7806853792048036
-1.9629490375518799 -2.6018996949977033
-1.607563853263855 -2.6485974179207803
-0.712519109249115 -1.8754676674992967
-13.4399995803833 -12.73644509481852
-1.0986167192459106 -1.74656407412508
-1.2477909326553345 -2.353958614694739
-0.8858245015144348 -1.8501158060180405
-1.5417516231536865 -3.4210942344646282
-1.5176204442977905 -3.2767450982091164
-10.260000228881836 -9.586265939146628
-9.5600004196167 -2.3511872364797197
-0.9647989273071289 -1.9931240842842266
-1.1732038259506226 -1.7237458500805936
-0.9362803101539612 -2.4931255453919112
-1.5198142528533936 -3.2397988227504557
-2.4653143882751465 -3.0146915894574056
-2.5048017501831055 -3.014692132864899
-0.7256816029548645 -1.8188473237514642
-1.3355402946472168

solvent split

In [ ]:
del dataset
dataset = pickle_load("/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer/dataset.pkl")

In [ ]:
possible_solvents = ['hexane', 'water', 'acetone', 'ethanol', 'benzene', 'ethylacetate',
               'dichloromethane', 'acetonitrile', 'thf', 'dmso', 'dmf', 'octanol', 'hexadecane', 'cyclohexane']

p_s= 'cyclohexane'
args.solvent_split=p_s
trainset, valset, testset = solvent_split(
    dataset, args.solvent_split, random_seed=args.random_seed)

In [ ]:
# Scale training dataset features
if args.feature_scaling:
    solute_features_scaler,solvent_features_scaler= trainset.normalize_features()
    #solute_features_scaler, solvent_features_scaler = trainset.normalize_features()
    valset.normalize_features(solute_features_scaler, solvent_features_scaler)
    testset.normalize_features(solute_features_scaler, solvent_features_scaler)
    #testset.normalize_features(solute_features_scaler)
else:
    solute_features_scaler, solvent_features_scaler = None, None

label_scaler = trainset.normalize_labels()
if not args.distributed or (args.distributed and args.gpu == 0):
    torch.save(dataset.state_dict(), os.path.join(args.save_dir, args.dataset_state_dict_filename))
    print(
        "Trainset size: {}, valset size: {}: testset size: {}.".format(
            len(trainset), len(valset), len(testset)
        )
    )
bs = max(len(valset) // 10, 1)
val_loader = DataLoaderSolvation(valset, batch_size=bs, shuffle=False)
bs = max(len(testset) //10, 1)
test_loader = DataLoaderSolvation(testset, batch_size=bs, shuffle=False)
### model
feature_names = ["atom", "bond", "global"]
solute_feature_names = ["atom", "bond","atom2", "bond2", "global"]
set2set_ntypes_direct = ["global"]
solute_feature_size = dataset.feature_sizes[0]
# solute_feature_size ={'bond': 11,'atom': 28,'atom2': 28, 'bond2': 11, 'global': 3}
solute_feature_size = {'bond': 11, 'atom': 30, 'atom2': 30, 'bond2': 11, 'global': 3}

solvent_feature_size = dataset.feature_sizes[1]
args.solute_feature_size = solute_feature_size
args.solvent_feature_size = solvent_feature_size
args.set2set_ntypes_direct = set2set_ntypes_direct


In [ ]:
model = GatedGCNSolvationNetwork(
        solute_in_feats=args.solute_feature_size,
        solvent_in_feats=args.solvent_feature_size,
        embedding_size=args.embedding_size,
        gated_num_layers=args.gated_num_layers,
        gated_hidden_size=args.gated_hidden_size,
        gated_num_fc_layers=args.gated_num_fc_layers,
        gated_graph_norm=args.gated_graph_norm,
        gated_batch_norm=args.gated_batch_norm,
        gated_activation=args.gated_activation,
        gated_residual=args.gated_residual,
        gated_dropout=args.gated_dropout,
        num_lstm_iters=args.num_lstm_iters,
        num_lstm_layers=args.num_lstm_layers,
        set2set_ntypes_direct=args.set2set_ntypes_direct,
        fc_num_layers=args.fc_num_layers,
        fc_hidden_size=args.fc_hidden_size,
        fc_batch_norm=args.fc_batch_norm,
        fc_activation=args.fc_activation,
        fc_dropout=args.fc_dropout,
        outdim=1,
        conv="GatedGCNConv",
    )
optimizer = torch.optim.Adam(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay
)
loss_func = MSELoss(reduction="mean")
metric = L1Loss(reduction="sum")

scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.4, patience=50, verbose=True
)
stopper = EarlyStopping(patience=150)

state_dict_objs = {"model": model, "optimizer": optimizer, "scheduler": scheduler}
model.to(args.gpu)

In [ ]:
args.save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/exp"
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
        state_dict_objs, map_location=loc, save_dir=args.save_dir, filename="best_checkpoint.pkl")

In [ ]:
test_acc = evaluate(model, feature_names,solute_feature_names, test_loader, metric, label_scaler, args.gpu)
y_true, y_pred = evaluate(model, feature_names,solute_feature_names, test_loader, metric, 
                                label_scaler, args.gpu, return_preds=True)

print(p_s)
print(len(y_pred))
print("\n#Test MAE: {:12.6e} \n".format(test_acc))
print("\n#Test RMSE: {:12.6e} \n".format(mean_squared_error(y_true, y_pred, squared=False)))

In [ ]:
print(y_pred)
print(testset.indices)

In [ ]:
import pandas as pd

# 假设这些变量已经定义好
csv_path = '/home/1739678401/2024.9dataclean/MMGNN_0901/data/exp_data.csv'
output_filename = f'{args.solvent_split}.csv'  # 输出文件名
indices = testset.indices     # 你的编号列表（如 [626, 102, 54] 等）
predictions = y_pred  # 与编号列表对应的预测结果

# 读取CSV文件
df = pd.read_csv(csv_path)


rows = []
for idx, pred in zip(indices, predictions):
    true_idx = idx 
    row = df.iloc[true_idx]
    rows.append([
        row.iloc[0],  # 第一列
        row.iloc[1],  # 第二列
        row.iloc[4],  # 第五列（索引从0开始）
        pred          # 预测值
    ])

# 创建新的DataFrame
output_df = pd.DataFrame(rows, columns=['smiles1', 'smiles2', 'd', 'Prediction'])

# 保存为CSV
output_df.to_csv(output_filename, index=False)


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import os

# 定义可能的溶剂文件名（假设文件后缀为.csv）
possible_solvents = [
    'hexane', 'water', 'acetone', 'ethanol', 'benzene', 'ethylacetate',
    'dichloromethane', 'acetonitrile', 'thf', 'dmso', 'dmf',
    'octanol', 'hexadecane', 'cyclohexane'
]

# 遍历所有csv文件
for solvent in possible_solvents:
    filename = f"{solvent}.csv"
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        if 'smiles2' not in df.columns:
            print(f"File {filename} does not contain 'smiles2' column. Skipping.")
            continue

        # 计算分子量
        def compute_mol_weight(smiles):
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                return Descriptors.MolWt(mol)
            else:
                return None

        df['MolecularWeight'] = df['smiles2'].apply(compute_mol_weight)

        # 保存文件（覆盖原文件）
        df.to_csv(filename, index=False)
        print(f"Updated file: {filename}")
    else:
        print(f"File {filename} not found.")


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors

# 创建分子对象
mol = Chem.MolFromSmiles('CCO')

# 计算相对分子质量
mol_weight = Descriptors.MolWt(mol)

print(f"CCO 的相对分子质量是: {mol_weight:.4f}")


element split

In [ ]:
del dataset
dataset = pickle_load("/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer/dataset.pkl")

In [ ]:
possible_elems = ['Br', 'Cl', 'F', 'I', 'N', 'O', 'S']
p_e='S'
args.element_split=p_e
trainset, valset, testset = element_split(dataset, args.element_split, random_seed=args.random_seed)

In [ ]:
# Scale training dataset features
if args.feature_scaling:
    solute_features_scaler,solvent_features_scaler= trainset.normalize_features()
    #solute_features_scaler, solvent_features_scaler = trainset.normalize_features()
    valset.normalize_features(solute_features_scaler, solvent_features_scaler)
    testset.normalize_features(solute_features_scaler, solvent_features_scaler)
    #testset.normalize_features(solute_features_scaler)
else:
    solute_features_scaler, solvent_features_scaler = None, None

label_scaler = trainset.normalize_labels()
if not args.distributed or (args.distributed and args.gpu == 0):
    torch.save(dataset.state_dict(), os.path.join(args.save_dir, args.dataset_state_dict_filename))
    print(
        "Trainset size: {}, valset size: {}: testset size: {}.".format(
            len(trainset), len(valset), len(testset)
        )
    )
bs = max(len(valset) // 10, 1)
val_loader = DataLoaderSolvation(valset, batch_size=bs, shuffle=False)
bs = max(len(testset) //10, 1)
test_loader = DataLoaderSolvation(testset, batch_size=bs, shuffle=False)
### model
feature_names = ["atom", "bond", "global"]
solute_feature_names = ["atom", "bond","atom2", "bond2", "global"]
set2set_ntypes_direct = ["global"]
solute_feature_size = dataset.feature_sizes[0]
# solute_feature_size ={'bond': 11,'atom': 28,'atom2': 28, 'bond2': 11, 'global': 3}
solute_feature_size = {'bond': 11, 'atom': 30, 'atom2': 30, 'bond2': 11, 'global': 3}

solvent_feature_size = dataset.feature_sizes[1]
args.solute_feature_size = solute_feature_size
args.solvent_feature_size = solvent_feature_size
args.set2set_ntypes_direct = set2set_ntypes_direct


In [ ]:
model = GatedGCNSolvationNetwork(
        solute_in_feats=args.solute_feature_size,
        solvent_in_feats=args.solvent_feature_size,
        embedding_size=args.embedding_size,
        gated_num_layers=args.gated_num_layers,
        gated_hidden_size=args.gated_hidden_size,
        gated_num_fc_layers=args.gated_num_fc_layers,
        gated_graph_norm=args.gated_graph_norm,
        gated_batch_norm=args.gated_batch_norm,
        gated_activation=args.gated_activation,
        gated_residual=args.gated_residual,
        gated_dropout=args.gated_dropout,
        num_lstm_iters=args.num_lstm_iters,
        num_lstm_layers=args.num_lstm_layers,
        set2set_ntypes_direct=args.set2set_ntypes_direct,
        fc_num_layers=args.fc_num_layers,
        fc_hidden_size=args.fc_hidden_size,
        fc_batch_norm=args.fc_batch_norm,
        fc_activation=args.fc_activation,
        fc_dropout=args.fc_dropout,
        outdim=1,
        conv="GatedGCNConv",
    )
optimizer = torch.optim.Adam(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay
)
loss_func = MSELoss(reduction="mean")
metric = L1Loss(reduction="sum")

scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.4, patience=50, verbose=True
)
stopper = EarlyStopping(patience=150)

state_dict_objs = {"model": model, "optimizer": optimizer, "scheduler": scheduler}
model.to(args.gpu)

In [ ]:
args.save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/exp"
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
        state_dict_objs, map_location=loc, save_dir=args.save_dir, filename="best_checkpoint.pkl")

In [ ]:
test_acc = evaluate(model, feature_names,solute_feature_names, test_loader, metric, label_scaler, args.gpu)
y_true, y_pred = evaluate(model, feature_names,solute_feature_names, test_loader, metric, 
                                label_scaler, args.gpu, return_preds=True)

print(p_e)
print(len(y_true))
print("\n#Test MAE: {:12.6e} \n".format(test_acc))
print("\n#Test RMSE: {:12.6e} \n".format(mean_squared_error(y_true, y_pred, squared=False)))

In [ ]:
args.save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/theory"
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
        state_dict_objs, map_location=loc, save_dir=args.save_dir, filename="best_checkpoint.pkl")

In [ ]:
test_acc = evaluate(model, feature_names,solute_feature_names, test_loader, metric, label_scaler, args.gpu)
y_true, y_pred = evaluate(model, feature_names,solute_feature_names, test_loader, metric, 
                                label_scaler, args.gpu, return_preds=True)

print(p_e)
print(len(y_true))
print("\n#Test MAE: {:12.6e} \n".format(test_acc))
print("\n#Test RMSE: {:12.6e} \n".format(mean_squared_error(y_true, y_pred, squared=False)))

In [ ]:
args.save_dir="/home/1739678401/2024.9dataclean/MMGNN_0901/result/transfer"
loc = "cuda:{}".format(args.gpu)
checkpoint = load_checkpoints(
        state_dict_objs, map_location=loc, save_dir=args.save_dir, filename="best_checkpoint.pkl")

In [ ]:
test_acc = evaluate(model, feature_names,solute_feature_names, test_loader, metric, label_scaler, args.gpu)
y_true, y_pred = evaluate(model, feature_names,solute_feature_names, test_loader, metric, 
                                label_scaler, args.gpu, return_preds=True)

print(p_e)
print(len(y_true))
print("\n#Test MAE: {:12.6e} \n".format(test_acc))
print("\n#Test RMSE: {:12.6e} \n".format(mean_squared_error(y_true, y_pred, squared=False)))